<a href="https://colab.research.google.com/github/ldeluigi/supermarket-2077-product-vision/blob/master/ProductDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminary Operations

## Download datasets

In [ ]:
!rm -rf sample_data
!gdown --id 1fDr4g4wbnSRkuCYyS3wpuJS7Ax22bVB_ -O all.zip
!unzip -oq all.zip

%matplotlib inline

## Imports

In [ ]:
import scipy.io
import os
from pathlib import Path
import re
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import itertools
import shutil
from tqdm.notebook import tqdm
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cosine
from statistics import mode

# Data Visualization Utilities

In [ ]:
def show_image(img):
  plt.axis('off')
  plt.imshow(img)

def show_grayscale_image(img):
  show_image(cv2.merge([img, img, img]))

def plot_grid(images, columns, show_axis=False, labels=None):
  if len(images) == 0 or columns <= 0:
    return
  height = 1 + math.ceil(len(images) / columns) * 2
  width = columns * 4
  dpi = max(images[0].shape[0], images[0].shape[1]) // 2
  fig = plt.figure(figsize=(width, height), dpi=dpi)
  fig.subplots_adjust(hspace=0.4)
  for index, img in enumerate(images, start=1):
    if 'float' in img.dtype.str:
      img = (img * 255).astype('uint8')
    sp = fig.add_subplot(math.ceil(len(images) / columns), columns, index)
    if not show_axis:
      plt.axis('off')
    plt.imshow(img)
    if labels is not None:
      l = len(labels)
      sp.set_title(labels[(index-1) % l], fontsize=10)
    else:
      sp.set_title(index, fontsize=10)

def dataset_plot_grid(indexes, columns, dataset, draw_item):
  fig = plt.figure(figsize=(12, 6), dpi=120)
  # fig.subplots_adjust(hspace=0.2)
  for index, i_img in enumerate(indexes, start=1):
    sp = fig.add_subplot(math.ceil(len(indexes) / columns), columns, index)
    row = dataset[i_img]
    draw_item(row, sp)

# Raw image loading

## Utilities to read raw data from disk

In [ ]:
training_dirname = 'Training'

def create_class_label(class_index, class_name):
  return class_name

def read_classes():
  mat = scipy.io.loadmat(os.path.join(training_dirname, 'TrainingClassesIndex.mat'))
  raw_classes = list(map(lambda x: x[0], mat['classes'][0]))
  classes = map(lambda x: (x[0], create_class_label(*x)), enumerate(raw_classes, start=1))
  return dict(classes), dict(enumerate(raw_classes, start=1))

def read_training_data(classes):
  result = []
  for class_index, class_name in classes.items():
    dirname_images = os.path.join(training_dirname, class_name)
    directory_images = os.fsencode(dirname_images)
    for file in os.listdir(directory_images):
      img = cv2.imread(os.path.join(dirname_images, os.fsdecode(file)))
      img_rgb =  cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      result.append((img_rgb, class_index))
  return np.rec.array(result, dtype=[('image', 'O'), ('class_index', 'i4')])

def read_store_data(storename):
  dirname_anno = os.path.join(storename, 'annotation')
  dirname_images = os.path.join(storename, 'images')
  directory_anno = os.fsencode(dirname_anno)
  directory_images = os.fsencode(dirname_images)

  result = []

  for file in os.listdir(directory_anno):
    filename = os.fsdecode(file)
    if filename.endswith(".mat"): 
      mat = scipy.io.loadmat(os.path.join(dirname_anno, filename))
      number = re.search(r'^anno.(\d+).mat$', filename).group(1)
      img = cv2.imread(os.path.join(dirname_images, number + '.jpg'))

      img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img_annotation = mat['annotation'][0, 0]
      bboxes = map(lambda x: x[0], img_annotation[0][0])
      labels = map(lambda x: str(x[0][0][0]), img_annotation[1][0])
      class_indexes = img_annotation[2][0]
      result.append((img_rgb, list(zip(bboxes, labels, class_indexes))))
  return np.rec.array(result, dtype=[('image', 'O'), ('items', 'O')])

## Prepare products class dictionary

In [ ]:
classes, raw_classes = read_classes()

def class_name(class_index):
  return classes[class_index] if class_index >= 0 else None

## Load training raw images

In [ ]:
products = read_training_data(raw_classes)

## Products visualization

In [ ]:
def show_products_with_class(indexes, columns, dataset):
  def show_single_product_with_class(row, sp):
    plt.axis('off')
    plt.imshow(row.image)
    sp.set_title(class_name(row.class_index), fontsize=10)
  dataset_plot_grid(indexes, columns, dataset, show_single_product_with_class)

show_products_with_class(np.random.randint(0, len(products), 6), 3, products)

# Raw Image preprocessing

## Background removal

In [ ]:
# code taken from https://www.kaggle.com/vadbeg/opencv-background-removal and modified

def remove_background(img, threshold, use_mask=False):
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  _, threshed = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY_INV)

  kernel_size = round(max(img.shape[0], img.shape[1]) * 0.02)
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
  morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

  cnts = cv2.findContours(morphed, 
                          cv2.RETR_EXTERNAL,
                          cv2.CHAIN_APPROX_SIMPLE)[0] # should be [1] for cv2 version <= 4

  cnts = sorted(cnts, key=cv2.contourArea)

  mask = cv2.drawContours(threshed, [cnts[-1]], 0, [255], cv2.FILLED)

  x, y, w, h = cv2.boundingRect(cnts[-1])

  if use_mask:
    masked_data = cv2.bitwise_and(img, img, mask=mask)
    dst = masked_data[y: y + h, x: x + w]
    r, g, b = cv2.split(dst)
    alpha = mask[y: y + h, x: x + w]

    rgba = [r, g, b, alpha]
    dst = cv2.merge(rgba, 4)
  else:
    dst = img[y: y + h, x: x + w]

  return dst

n = 777
print(f'Index: {n}')
print(f'Class: {class_name(products[n].class_index)}')
plot_grid([products[n].image, remove_background(products[n].image, 250)], 2, show_axis=True)

## Image resize

In [ ]:
background_color = 255

def resize_image(img, size, color=[background_color, background_color, background_color]):
  target_w, target_h = size
  original_h, original_w, _ = img.shape
  target_ar = target_w / target_h
  original_ar = original_w / original_h

  scale_factor = target_h / original_h if target_ar > original_ar else target_w / original_w
  scaled_w = round(original_w * scale_factor)
  scaled_h = round(original_h * scale_factor)
  scaled_size = (scaled_w, scaled_h)
  resized = cv2.resize(img, scaled_size)

  delta_h = target_h - scaled_h
  delta_w = target_w - scaled_w
  top    = delta_h // 2
  left   = delta_w // 2
  bottom = delta_h - top
  right  = delta_w - left

  return cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

n = 1315
print(f'Index: {n}')
print(f'Class: {class_name(products[n].class_index)}')
image = products[n].image
image = remove_background(image, 250)
plot_grid([image, resize_image(image, (400, 400))], 2, show_axis=True)

## Dataset preparation



### Image cleaning

In [ ]:
def clean_image(img):
  threshold = 250
  img = remove_background(img, threshold)
  return img

### Prepare dataset

In [ ]:
all_products_images = []
for image, class_index in products:
  cleaned_image = clean_image(image)
  all_products_images.append(cleaned_image)

print(len(all_products_images))

# Computer vision model #1

#### Save dataset on disk in order to save memory with ImageDataGenerators
Optionally reduce the dataset items

In [ ]:
number_of_images_to_save = 1000
reduce_dataset = False

In [ ]:
train_data_directory = 'Temp'
shutil.rmtree(train_data_directory, ignore_errors=True)

index_to_class_map = dict()
def index_to_class(index):
  # We could search in products for O(n)
  # Instead we use a map for O(1)
  return index_to_class_map[index]

if reduce_dataset:
  products_size = number_of_images_to_save
  products_to_dump = np.random.choice(products.shape[0], number_of_images_to_save, replace = False)
else:
  products_size = products.shape[0]
  products_to_dump = np.arange(products_size)

for index, product_index in tqdm(enumerate(products_to_dump), total=products_size, desc='Writing files...'):
  image, class_index = products[product_index]
  index_str = f'{index:04d}'
  output_dir = os.path.join(train_data_directory, index_str)
  Path(output_dir).mkdir(parents=True, exist_ok=True)
  cleaned_image = clean_image(image)
  out = cv2.cvtColor(cleaned_image, cv2.COLOR_RGBA2BGRA)
  cv2.imwrite(os.path.join(output_dir, f'{index_str}.png'), out)
  index_to_class_map[index] = class_index

## Higher-order algorithms
These functions allow to plug-and-play feature extraction and classification and to measure their performances afterwards.

In [ ]:
def accuracy(extract_features, compute_feature_distance, threshold, n = 100):
  actual = []
  predicted = []
  false_positives = []
  false_negatives = []
  it = probability_merge(mismatched_images(), matched_images(), p = 0.5)
  for _ in tqdm(range(n), total=n, desc='Calculating accuracy...'):
    original, transformed, label = next(it)
    original_features = extract_features(original)
    transformed_features = extract_features(transformed)
    distance = compute_feature_distance(original_features, transformed_features)
    prediction = 1 if distance > threshold else 0
    actual.append(label)
    predicted.append(prediction)
    if label != prediction:
      (false_positives if prediction == 0 else false_negatives).append((original, transformed))
  confusion = confusion_matrix(actual, predicted)
  print(confusion)
  to_be_plotted = []
  for original, transformed in false_positives:
    to_be_plotted.append(original)
    to_be_plotted.append(transformed)
  plot_grid(to_be_plotted, 2, labels=['Original', 'False positive'])

  to_be_plotted = []
  for original, transformed in false_negatives:
    to_be_plotted.append(original)
    to_be_plotted.append(transformed)
  plot_grid(to_be_plotted, 2, labels=['Original', 'False negative'])

Prepare (or reset, if rerun) feature cache/database to be queried when classifying

In [ ]:
feature_cache = None
feature_cache_name = None

In [ ]:
def cache_features(extract_features):
  global feature_cache, feature_cache_name
  if extract_features.__name__ != feature_cache_name:
    feature_cache_name = None
  if feature_cache is None or feature_cache_name is None:
    feature_cache_name = extract_features.__name__
    feature_cache = []
    for image, index in tqdm(single_images(single_pass = True), desc = 'Creating feature cache...'):
      features = extract_features(image)
      feature_cache.append((features, index))
  return feature_cache

def product_classification(img, extract_features, compute_feature_distance):
  img_features = extract_features(img)
  feature_db = cache_features(extract_features)
  distances_iter = map(lambda fc: compute_feature_distance(fc[0], img_features), feature_db)
  index, best_distance = min(enumerate(distances_iter), key=lambda x:x[1])
  best_match_label = feature_db[index][1]
  best_match_index = int(best_match_label)
  class_index = index_to_class(best_match_index)
  return best_match_index, class_index, class_name(class_index), best_distance

def product_classifications(imgs, extract_features, compute_feature_distance):
  return list(map(lambda im: product_classification(im, extract_features, compute_feature_distance), imgs))

## Model definition


### Feature extraction

In [ ]:
def rgb_image_to_feature_channels(img):
  img_hsv = cv2.cvtColor((img * 255).astype('uint8'), cv2.COLOR_RGB2LAB)
  return list(cv2.split(img_hsv))
  
def create_feature_extractor():
  orb = cv2.ORB_create(edgeThreshold=20, nfeatures=100) # https://docs.opencv.org/master/d1/d89/tutorial_py_orb.html
  return orb

feature_extractor = create_feature_extractor()

def extract_features_raw(img):
  channels = rgb_image_to_feature_channels(img)
  return list(map(lambda x: feature_extractor.detectAndCompute(x, None), channels))

def extract_features(img, blur_size = 5):
  blurred_img = cv2.GaussianBlur(img, (blur_size, blur_size), 1) if blur_size > 1 else img
  return extract_features_raw(blurred_img)

### Feature comparison

In [ ]:
def create_bf_matcher():
  return cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

def create_flann_matcher():
  FLANN_INDEX_LSH = 6
  index_params = dict(
    algorithm = FLANN_INDEX_LSH,
    table_number = 6,     # 12
    key_size = 12,        # 20
    multi_probe_level = 1 # 2
  )
  search_params = dict(checks = 50)
  return cv2.FlannBasedMatcher(index_params, search_params)

In [ ]:
def get_keypoint_matches(p, t):
  k_1, d_1 = p
  k_2, d_2 = t
  if d_1 is None or d_2 is None:
    return []
  matcher = create_bf_matcher()
  #matcher = create_flann_matcher()
  return matcher.match(d_1, d_2)

def get_matches_distances(predictions, targets):
  distances = []
  for p, t in zip(predictions, targets):
    matches = get_keypoint_matches(p, t)
    n_matches = len(matches)
    if n_matches == 0:
      distance = 1e6
    else:
      # 1 / sum(map(lambda m: 1 / (m.distance ** 2 + epsilon), matches))
      distance = sum(filter(lambda d: True, map(lambda m: m.distance, matches))) / n_matches
    distances.append(distance)
  return distances

def cumulative_distance(predictions, targets):
  d = get_matches_distances(predictions, targets)
  return d[0] ** 2 + d[1] ** 2 + d[2] ** 2

In [ ]:
def show_image_comparison(first_img, second_img):
  channels_1 = rgb_image_to_feature_channels(first_img)
  channels_2 = rgb_image_to_feature_channels(second_img)
  f1 = extract_features(first_img)
  f2 = extract_features(second_img)
  matcher = create_flann_matcher()
  
  image_grid = []
  first_image_int = (first_img * 255).astype('uint8')
  second_image_int = (second_img * 255).astype('uint8')
  for c1, c2, (k1, d1), (k2, d2) in zip(channels_1, channels_2, f1, f2):
    image_grid.append(cv2.drawKeypoints(first_image_int, k1, None))
    image_grid.append(cv2.drawKeypoints(second_image_int, k2, None))
    matches = matcher.match(d1, d2)
    matches = sorted(matches, key = lambda x: x.distance)
    image_grid.append(cv2.drawMatches(c1, k1, c2, k2, matches, None, flags=2))
  plot_grid(image_grid, 3, labels=['Original', 'Altered', 'Matches on LAB'])

show_image_comparison(*next(matched_images()))
show_image_comparison(*next(mismatched_images()))

In [ ]:
def stats(it, n = 1):
  results = []
  for _ in range(n):
    original, transformed = next(it)
    original_f = extract_features(original)
    transformed_f = extract_features(transformed)
    if original_f[1] is None or transformed_f[1] is None:
      continue
    results.append(cumulative_distance(original_f, transformed_f))
  print('\tAvg:', np.mean(results))
  print('\tMedian:', np.median(results))
  print('\tMax:', np.max(results))
  print('\tMin:', np.min(results))
  print('\tStdev:', np.std(results))

print("Summary of matched images feature comparison by comulative distance:")
stats(matched_images(), 500)
print("Summary of mismatched images feature comparison by cumulative distance:")
stats(mismatched_images(), 500)

### Feature classification

In [ ]:
# Create data on which we must find a discriminator
def find_discriminator(n = 1000):
  X = []
  y = []
  Xv = []
  yv = []
  matched = matched_images()
  mismatched = mismatched_images()
  for i in range(n):
    if i < n / 2:
      im1, im2 = next(matched)
      X.append(get_matches_distances(extract_features(im1), extract_features(im2)))
      y.append(0)
      im1, im2 = next(matched)
      Xv.append(get_matches_distances(extract_features(im1), extract_features(im2)))
      yv.append(0)
    else:
      im1, im2 = next(mismatched)
      X.append(get_matches_distances(extract_features(im1), extract_features(im2)))
      y.append(1)
      im1, im2 = next(mismatched)
      Xv.append(get_matches_distances(extract_features(im1), extract_features(im2)))
      yv.append(1)
  perm = np.random.permutation(len(X))
  X = np.asarray(X)[perm]
  y = np.asarray(y)[perm]
  perm = np.random.permutation(len(X))
  Xv = np.asarray(Xv)[perm]
  yv = np.asarray(yv)[perm]
  lr = LogisticRegression()
  clf = lr.fit(X, y)
  score = lr.score(Xv, yv)
  print(f"Discriminator accuracy: {score}")
  return clf

classifier = find_discriminator(400)

def classify_distances(distances):
  return classifier.predict_proba([distances])[0][1]

def classify_features(predictions, targets):
  return classify_distances(get_matches_distances(predictions, targets))

In [ ]:
def stats(it, n = 1):
  results = []
  for _ in range(n):
    original, transformed = next(it)
    original_f = extract_features(original)
    transformed_f = extract_features(transformed)
    if original_f[1] is None or transformed_f[1] is None:
      continue
    results.append(classify_features(original_f, transformed_f))
  print('\tAvg:', np.mean(results))
  print('\tMedian:', np.median(results))
  print('\tMax:', np.max(results))
  print('\tMin:', np.min(results))
  print('\tStdev:', np.std(results))

print("Summary of matched images feature comparison by discrimination with probability:")
stats(matched_images(), 500)
print("Summary of mismatched images feature comparison by discrimination with probability:")
stats(mismatched_images(), 500)

### Evaluate performance

In [ ]:
accuracy(extract_features, cumulative_distance, 10000, n = 400)

In [ ]:
def column(M, c):
  return [row[c] for row in M]
test_size = 10
test_images = list(itertools.islice(single_altered_images(), test_size))
predictions = product_classifications(column(test_images, 0), extract_features, cumulative_distance)
cm = confusion_matrix(column(test_images, 1), column(predictions, 0))
print("Accuracy: ", np.trace(cm) / test_size)
plot_grid(column(test_images[:20], 0), 5, labels=column(predictions[:20], 2))
del test_images, predictions, cm

In [ ]:
import pandas
print(pandas.DataFrame(np.fromiter(map(lambda e: len(e[0][0][0]), feature_cache), dtype='uint32')).describe())
del pandas

# Computer vision model #2

In [ ]:
def extract_features_from_image(img, extractor):
  keypoints, descriptors = extractor.detectAndCompute(img, None)
  if keypoints is None or descriptors is None:
    return [], []
  return keypoints, descriptors

def extract_features(images):
  orb = cv2.ORB_create(nfeatures=150)
  return list(map(lambda img: extract_features_from_image(img, orb), images))

features = extract_features(all_products_images)
descriptors = list(map(lambda f: f[1], features))
n = 0
print("Numero di keypoint estratti dall'immagine", n, "=", len(descriptors[0]))
print("Numero di descrittori per ogni keypoint =", len(descriptors[0][0]))
print("Numero di descrittori estratti in media da ogni immagine", np.mean(list(map(lambda d: len(d), descriptors))))

In [ ]:
def create_bovw(features, n_clusters):
  kmeans = KMeans(n_clusters = n_clusters, n_init = 1, max_iter = 500, verbose = 1)
  all_descriptors = []
  for _, desc in features:
    all_descriptors.extend(desc)
  print('Running KMeans on', len(all_descriptors), 'patterns')
  kmeans.fit(all_descriptors)
  print(kmeans.labels_)
  i = 0
  labels = []
  for _, desc in features:
    curr_label = []
    for d in desc:
      curr_label.append(kmeans.labels_[i])
      i += 1
    labels.append(curr_label)
  return (kmeans.cluster_centers_, labels)

n_clusters = 128
bovw, bovw_labels = create_bovw(features, n_clusters)


print("Numero di visual words =", len(bovw))
print("Numero di descrittori per ogni visual word =", len(bovw[0]))

In [ ]:
def normalize_histogram(histogram):
  sum = np.sum(histogram)
  if sum == 0:
    return [0.0] * len(histogram)
  return [x / sum for x in histogram]

def create_histogram_from_labels(n, labels):
  histogram = [0] * n
  for l in labels:
    histogram[l] += 1
  return normalize_histogram(histogram)

histograms = [create_histogram_from_labels(n_clusters, l) for l in bovw_labels]

In [ ]:
n = np.random.randint(0, len(all_products_images))
img = all_products_images[n]
keypoints, _ = features[n]
labels = bovw_labels[n]
keypoints_by_label = {}

for k, l in zip(keypoints, labels):
  if l not in keypoints_by_label:
    keypoints_by_label[l] = []
  keypoints_by_label[l].append(k)

for l in keypoints_by_label:
  [r, g, b] = list(np.random.choice(range(256), size=3))
  cv2.drawKeypoints(img, keypoints_by_label[l], img, color = (int(b), int(g), int(r)))

print("Numero di keypoint =", len(keypoints))
show_image(img)

In [ ]:
def create_histogram_for_image(img, descriptors, bovw, distance_metric):
  #[hist_b, hist_g, hist_r] = cv2.calcHist([img], [0, 1, 2], mask=None, [256], [0, 256])
  histogram = [0] * len(bovw)
  for d in descriptors:
    nearest = 0
    nearest_distance = distance_metric(d, bovw[0])
    for i in range(1, len(bovw)):
      vw = bovw[i]
      distance = distance_metric(d, vw)
      if distance < nearest_distance:
        nearest = i
        nearest_distance = distance
    histogram[nearest] += 1
  return normalize_histogram(histogram)

def cosine_distance(x, y):
  return cosine(x, y)

def euclidean_distance(x, y):
  return np.linalg.norm(x - y)

def create_histograms(images, features, bovw, show_progress=False):
  histograms = []
  iterator = zip(images, features)
  if show_progress:
    iterator = tqdm(iterator, total = len(images), desc = "calculating histograms...")
  for image, f in iterator:
    _, descriptors = f
    histogram = create_histogram_for_image(image, descriptors, bovw, euclidean_distance)
    histograms.append(histogram)
  return histograms

In [ ]:
nn = NearestNeighbors(n_neighbors=1).fit(histograms)

In [ ]:
def predict(images, bovw, nn):
  image_features = extract_features(images)
  histograms = create_histograms(images, image_features, bovw)
  distances, indices = nn.kneighbors(histograms)
  distances = list(map(lambda x: x[0], distances))
  indices = list(map(lambda x: x[0], indices))
  return indices, distances

# Testing model on stores

In [ ]:
store = read_store_data('store2')

In [ ]:
def sliding_window(img, size, stride=(0.5, 0.5)):
  scale_values=[0.5, 0.3]
  height, width, _ = img.shape
  aspect_ratio = size[1] / size[0]
  for scale in scale_values:
    scaled_width = int(width * scale)
    scaled_height = int(height * scale)
    scaled_img = resize_image(img, (scaled_width, scaled_height))
    y_min = 0
    while y_min + size[1] < scaled_height:
      x_min = 0
      while x_min + size[0] < scaled_width:
        yield scaled_img[y_min : y_min + size[0] - 1, x_min : x_min + size[1] - 1]
        x_min += int(size[0] * stride[0])
      y_min += int(size[1] * stride[1])

size = (224, 224)
n = 20
img, _ = store[n]
sw_images = list(sliding_window(img, size))
indices, _ = predict(sw_images, bovw, nn)
predicted_classes = map(lambda i : products["class_index"][i], indices)
non_background = list(filter(lambda c : c != 1, predicted_classes))

target_class = mode(non_background)
print(class_name(target_class))
show_image(img)

In [ ]:
test_tea = b'...'
import base64
with open("tea.png", "wb") as fh:
    fh.write(base64.decodebytes(test_tea))
img = cv2.cvtColor(cv2.imread("tea.png"), cv2.COLOR_BGR2RGB)
indices, _ = predict([img], bovw, nn)
predicted_class = products["class_index"][indices[0]]
print(class_name(predicted_class))
show_image(img)

## Performance evaluation


In [ ]:
from sklearn.utils.extmath import weighted_mode
from scipy.stats import mode

def predict_with_window(img):
  sw_images = list(sliding_window(img, (img.shape[0] // 3, img.shape[1] // 3)))
  found_indices, distances = predict(sw_images, bovw, nn)
  found_classes = map(lambda i : products["class_index"][i], found_indices)
  ignore_background_classes, ignore_background_distances = zip(*filter(lambda c : c[0] != 1, zip(found_classes, distances)))
  ignore_background_weights = [1/x for x in ignore_background_distances]
  most_voted_class = weighted_mode(ignore_background_classes, ignore_background_weights)[0][0]
  return most_voted_class
predictions = [predict_with_window(img[0]) for img in store]
score = [1 if t[1] == mode([x[2] for x in t[0][1]])[0] else 0 for t in zip(store, predictions)]
print("Accuracy:", sum(score) / len(score))

# Data Augmentation
Augmentation, fetching from disk...

## 3D rotation

In [ ]:
def image_3D_rotation(img, theta = 0, phi = 0, gamma = 0, dx = 0, dy = 0, dz = 0):
  """
  Parameters:
      img       : the image data as numpy array
      theta     : rotation around the x axis
      phi       : rotation around the y axis
      gamma     : rotation around the z axis (basically a 2D rotation)
      dx        : translation along the x axis
      dy        : translation along the y axis
      dz        : translation along the z axis (distance to the image)
  Output:
      image     : the rotated image
  
  Reference:
      1.        : http://stackoverflow.com/questions/17087446/how-to-calculate-perspective-transform-for-opencv-from-rotation-angles
      2.        : http://jepsonsblog.blogspot.tw/2012/11/rotation-in-3d-using-opencvs.html
      3.        : Code taken from https://github.com/eborboihuc/rotate_3d/blob/master/image_transformer.py
  """
  def deg_to_rad(deg):
    return deg * math.pi / 180.0
  def get_M(theta, phi, gamma, dx, dy, dz, size, focal):
    w = size[0]
    h = size[1]
    f = focal
    # Projection 2D -> 3D matrix
    A1 = np.array([ [1, 0, -w/2],
                    [0, 1, -h/2],
                    [0, 0, 1],
                    [0, 0, 1]])
    # Rotation matrices around the X, Y, and Z axis
    RX = np.array([ [1, 0, 0, 0],
                    [0, np.cos(theta), -np.sin(theta), 0],
                    [0, np.sin(theta), np.cos(theta), 0],
                    [0, 0, 0, 1]])
    RY = np.array([ [np.cos(phi), 0, -np.sin(phi), 0],
                    [0, 1, 0, 0],
                    [np.sin(phi), 0, np.cos(phi), 0],
                    [0, 0, 0, 1]])
    RZ = np.array([ [np.cos(gamma), -np.sin(gamma), 0, 0],
                    [np.sin(gamma), np.cos(gamma), 0, 0],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])
    # Composed rotation matrix with (RX, RY, RZ)
    R = np.dot(np.dot(RX, RY), RZ)
    # Translation matrix
    T = np.array([  [1, 0, 0, dx],
                    [0, 1, 0, dy],
                    [0, 0, 1, dz],
                    [0, 0, 0, 1]])
    # Projection 3D -> 2D matrix
    A2 = np.array([ [f, 0, w/2, 0],
                    [0, f, h/2, 0],
                    [0, 0, 1, 0]])
    # Final transformation matrix
    return np.dot(A2, np.dot(T, np.dot(R, A1)))
  height = img.shape[0]
  width = img.shape[1]
  num_channels = img.shape[2]
  rtheta = deg_to_rad(theta)
  rphi = deg_to_rad(phi)
  rgamma = deg_to_rad(gamma)
  d = np.sqrt(height**2 + width**2)
  focal = d / (2 * np.sin(rgamma) if np.sin(rgamma) != 0 else 1)
  dz = focal
  mat = get_M(rtheta, rphi, rgamma, dx, dy, dz, (width, height), focal)
  return cv2.warpPerspective(
    img.copy(),
    mat,
    (width, height),
    borderValue=(background_color, background_color, background_color),
    borderMode=cv2.BORDER_CONSTANT
  )

def random_spatial_rotation(theta_range, phi_range, gamma_range):
  return lambda img: image_3D_rotation(
    img, 
    theta = np.random.randint(theta_range[0], theta_range[1] + 1),
    phi = np.random.randint(phi_range[0], phi_range[1] + 1),
    gamma = np.random.randint(gamma_range[0], gamma_range[1] + 1)
  )

## Data generator parameters definition

In [ ]:
real_datagen = ImageDataGenerator(
    data_format = 'channels_last',
)

augmented_datagen = ImageDataGenerator(
    brightness_range = [0.5, 1.2],
    width_shift_range = size[0] // 10,
    height_shift_range = size[1] // 10,
    zoom_range = 0.1,
    fill_mode = 'constant',
    cval = background_color,
    data_format = 'channels_last',
    preprocessing_function = random_spatial_rotation(
      theta_range = (-20, 20),
      phi_range = (-30, 30),
      gamma_range = (-10, 10)
    )
)

def create_flow(single_pass = False):
  rd = real_datagen.flow_from_directory(
    directory = train_data_directory,
    target_size = size,
    color_mode = 'rgb',
    class_mode = 'sparse',
    batch_size = 1,
    shuffle = True
  )
  if single_pass:
    return itertools.islice(rd, rd.samples)
  return rd

def flow_to_augmented_tuple(t):
  rescale = 1./255
  original = t[0][0] * rescale
  transformed = augmented_datagen.preprocessing_function(t[0][0])
  transformed = augmented_datagen.random_transform(transformed) * rescale
  label = t[1][0]
  return original, transformed, label

def flow_to_tuple(t):
  rescale = 1./255
  original = t[0][0] * rescale
  label = t[1][0]
  return original, label

def mismatched_images():
  flow_1 = create_flow()
  flow_2 = create_flow()
  while True:
    t_1 = next(flow_1)
    t_2 = next(flow_2)
    if t_1[1][0] != t_2[1][0]:
      image, _ = flow_to_tuple(t_1)
      _, transformed, _ = flow_to_augmented_tuple(t_2)
      yield image, transformed
    else:
      next(flow_1) # used for misaligning iterators

def matched_images(single_pass = False):
  flow = create_flow(single_pass)
  for original, transformed, _ in map(flow_to_augmented_tuple, flow):
    yield original, transformed

def single_images(single_pass = False):
  flow = create_flow(single_pass)
  return map(flow_to_tuple, flow)

def single_altered_images(single_pass = False):
  flow = create_flow(single_pass)
  def tuple_f(t):
    _, transformed, label = flow_to_augmented_tuple(t)
    return transformed, label
  return map(tuple_f, flow)

res = []
it = matched_images()
for _ in range(10):
  t = next(it)
  res.append(t[0])
  res.append(t[1])

plot_grid(res, 2, labels=["Original", "Same, altered"])

res = []
it = mismatched_images()
for _ in range(10):
  t = next(it)
  res.append(t[0])
  res.append(t[1])

plot_grid(res, 2, labels=["Original", "Other, altered"])
del res, it

In [ ]:
def probability_merge(it_1, it_2, p = 0.5):
  while True:
    rand = np.random.random()
    it, label = (it_1, 1) if rand < p else (it_2, 0)
    original, transformed = next(it)
    yield original, transformed, label

it = probability_merge(mismatched_images(), matched_images(), p = 0.5)
res = []
for _ in range(20):
  t = next(it)
  res.append(t[0])
  res.append(t[1])

plot_grid(res, 2, labels=["First", "Second"])
del res, it